<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/ScraptruffleshuffleLoadMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.H4dW5Gghwf/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.J0esfnLDgW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.FNzPUiS49j/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd
import multiprocessing as mp
from selenium.webdriver.support.ui import WebDriverWait
import time

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [2]:
url = "https://www.truffleshuffle.co.uk/"
content = requests.get(url).content
soup = BeautifulSoup(content, 'html.parser')
navigation = soup.find('ul', class_='layoutInner')
link_tag = navigation.find_all('a')
all_category_link = []
for link in link_tag:
  base_link = "https://www.truffleshuffle.co.uk"
  full_link = base_link + link['href']
  all_category_link.append(full_link)
len(all_category_link)

37

In [3]:
import json
def post_character(image, title_head,  final_description):
  try:
    description = title_head + " " + final_description
    url = 'https://jormungandr.wizewerx.tech/eat'
    headers = {
        'Authorization': 'p7RF3TwoFtaQ9ymoFMS3ZdYfSdc7hEoYedXxZ1L5l7vSXf59ntjvZTymoF5rfpy8RSpD4IErZt2ykSv64Xbu86l7',
        'Content-Type': 'application/json'
    }

    data = {
        "topic": "fuk",
        "description": description,
        "payload": {},
        "files": image
    }
    json_data = json.dumps(data)
    response = requests.post(url, headers=headers, data=json_data)
    if response.status_code == 200:
        pass
    else:
        pass
  except:
    pass

In [6]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

def scrape_truffleshuffle(url):
    driver.get(url)

    try:
        # Wait until the "Load More" button is clickable
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "loadMore"))
        )

        # Scroll to the element
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)

        # Click the "Load More" button until it is no longer visible
        while load_more_button.is_displayed():
            try:
                load_more_button.click()
                time.sleep(2)
            except ElementClickInterceptedException:
                # Handle click interception, e.g., scroll again or wait for an overlay to disappear
                pass

        # Find all product cards
        card_product = driver.find_elements(By.CLASS_NAME, "isotopeElement.isotopeElementDefault.productListingBox")

        # Extract the product URLs
        product_urls = []
        for element in card_product:
            product_urls.append(element.get_attribute("href"))

        return product_urls
    except NoSuchElementException:
        return []


In [7]:
for cat in all_category_link:
  print(cat)

https://www.truffleshuffle.co.uk/new-in
https://www.truffleshuffle.co.uk/best-sellers
https://www.truffleshuffle.co.uk/movies
https://www.truffleshuffle.co.uk/tv-shows
https://www.truffleshuffle.co.uk/cartoons
https://www.truffleshuffle.co.uk/music
https://www.truffleshuffle.co.uk/gaming
https://www.truffleshuffle.co.uk/fairytales-and-books
https://www.truffleshuffle.co.uk/iconic-brands
https://www.truffleshuffle.co.uk/games-toys
https://www.truffleshuffle.co.uk/superheroes-and-villains
https://www.truffleshuffle.co.uk/horror
https://www.truffleshuffle.co.uk/icons
https://www.truffleshuffle.co.uk/cartoons/anime
https://www.truffleshuffle.co.uk/food-and-drink-brands
https://www.truffleshuffle.co.uk/everything-else
https://www.truffleshuffle.co.uk/t-shirts-and-clothing
https://www.truffleshuffle.co.uk/homewares
https://www.truffleshuffle.co.uk/accessories
https://www.truffleshuffle.co.uk/gifts
https://www.truffleshuffle.co.uk/nightwear-underwear
https://www.truffleshuffle.co.uk/games-puz

In [8]:
final_all_links = []
for index, link in enumerate(all_category_link):
    print(f"scrap index: {index} on this link: {link}")
    product_urls = scrape_truffleshuffle(link)
    final_all_links += product_urls

# Print the list of product URLs


# Close the driver
# driver.close()


scrap index: 0 on this link: https://www.truffleshuffle.co.uk/new-in
scrap index: 1 on this link: https://www.truffleshuffle.co.uk/best-sellers
scrap index: 2 on this link: https://www.truffleshuffle.co.uk/movies
scrap index: 3 on this link: https://www.truffleshuffle.co.uk/tv-shows
scrap index: 4 on this link: https://www.truffleshuffle.co.uk/cartoons


KeyboardInterrupt: ignored

In [ ]:
# link = "https://www.truffleshuffle.co.uk/movies"
# product_urls = scrape_truffleshuffle(link)
# for data in product_urls:
#   print(data)

In [ ]:
# len(product_urls)